## <span style="color:#0b486b">Assignment 02: Neural Embedding and Sequence Modelling</span>

## <span style="color:#0b486b">Set random seeds</span>

We start with importing tensorflow and numpy and setting random seeds for TF and numpy. You can use any seeds you prefer.

In [1]:
import numpy as np
import tensorflow as tf

tf.random.set_seed(123456)
np.random.seed(123456)

## <span style="color:#0b486b">Part 1: Download and preprocess the data</span>

The dataset we use for this assignment is a question classification dataset for which the train set consists of $5,500$ questions belonging to 6 coarse question categories including:
- abbreviation (ABBR), 
- entity (ENTY), 
- description (DESC), 
- human (HUM), 
- location (LOC) and 
- numeric (NUM).


Preprocessing data is an inital and important step in any machine learning or deep learning projects. The following *DataManager* class helps you to download data and preprocess data for the later steps of a deep learning project. 

In [2]:
import os
import zipfile
import collections
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
plt.style.use('ggplot')

class DataManager:
    def __init__(self, verbose=True, maxlen= 50, random_state=6789):
        self.verbose = verbose
        self.max_sentence_len = 0
        self.str_questions = list()
        self.str_labels = list()
        self.numeral_labels = list()
        self.maxlen = maxlen
        self.numeral_data = list()
        self.random_state = random_state
        self.random = np.random.RandomState(random_state)
        
    @staticmethod
    def maybe_download(dir_name, file_name, url, verbose= True):
        if not os.path.exists(dir_name):
            os.mkdir(dir_name)
        if not os.path.exists(os.path.join(dir_name, file_name)):
            urlretrieve(url + file_name, os.path.join(dir_name, file_name))
        if verbose:
            print("Downloaded successfully {}".format(file_name))
    
    def read_data(self, dir_name, file_names):
        for file_name in file_names:
            file_path= os.path.join(dir_name, file_name)
            self.str_questions= list(); self.str_labels= list()
            with open(file_path, "r", encoding="latin-1") as f:
                for row in f:
                    row_str= row.split(":")
                    label, question= row_str[0], row_str[1]
                    question= question.lower()
                    self.str_labels.append(label)
                    self.str_questions.append(question[0:-1])
                    if self.max_sentence_len < len(self.str_questions[-1]):
                        self.max_sentence_len= len(self.str_questions[-1])
         
        # turns labels into numbers
        le= preprocessing.LabelEncoder()
        le.fit(self.str_labels)
        self.numeral_labels = np.array(le.transform(self.str_labels))
        self.str_classes= le.classes_
        self.num_classes= len(self.str_classes)
        if self.verbose:
            print("\nSample questions... \n")
            print(self.str_questions[0:5])
            print("Labels {}\n\n".format(self.str_classes))
    
    def manipulate_data(self):
        tokenizer = tf.keras.preprocessing.text.Tokenizer()
        tokenizer.fit_on_texts(self.str_questions)
        self.numeral_data = tokenizer.texts_to_sequences(self.str_questions)
        self.numeral_data = tf.keras.preprocessing.sequence.pad_sequences(self.numeral_data, padding='post', truncating= 'post', maxlen= self.maxlen)
        self.word2idx = tokenizer.word_index
        self.word2idx = {k:v for k,v in self.word2idx.items()}
        self.idx2word = {v:k for k,v in self.word2idx.items()}
        self.vocab_size = len(self.word2idx)
    
    def train_valid_split(self, train_ratio=0.9):
        idxs = np.random.permutation(np.arange(len(self.str_questions)))
        train_size = int(train_ratio*len(idxs)) +1
        self.train_str_questions, self.valid_str_questions = self.str_questions[0:train_size], self.str_questions[train_size:]
        self.train_numeral_data, self.valid_numeral_data = self.numeral_data[0:train_size], self.numeral_data[train_size:]
        self.train_numeral_labels, self.valid_numeral_labels = self.numeral_labels[0:train_size], self.numeral_labels[train_size:]
        self.tf_train_set = tf.data.Dataset.from_tensor_slices((self.train_numeral_data, self.train_numeral_labels))
        self.tf_valid_set = tf.data.Dataset.from_tensor_slices((self.valid_numeral_data, self.valid_numeral_labels))

In [3]:
print('Loading data...')

dm = DataManager(maxlen=100)
dm.read_data("Data/", ["train_set.label"])   # read data

Loading data...

Sample questions... 

['manner how did serfdom develop in and then leave russia ?', 'cremat what films featured the character popeye doyle ?', "manner how can i find a list of celebrities ' real names ?", 'animal what fowl grabs the spotlight after the chinese year of the monkey ?', 'exp what is the full form of .com ?']
Labels ['ABBR' 'DESC' 'ENTY' 'HUM' 'LOC' 'NUM']




In [4]:
dm.manipulate_data()
dm.train_valid_split(train_ratio=0.6)

You now have a data manager, named *dm* containing the training and validiation sets in both text and numeric forms. Your task is to play around and read this code to figure out the meanings of some important attributes that will be used in the next parts.

#### <span style="color:red">**Question 1.1**</span> 
**What is the purpose of `self.train_str_questions` and `self.train_numeral_labels`? Write your code to print out the first five questions with labels in the training set.**

#Your answer here



**Ans:**
self.train_str_questions is a list that is used to store all the questions in the training set and self.train_numeral_labels is a list that is used to store all the labels of categories each of the quesions in the training data belongs to. It contains 6 labels for each of the data ABBR, ENTY etc.

In [5]:
#Your code here
for i in range(5):
    print('label: ',dm.train_numeral_labels[i],'\t','Question: ',dm.train_str_questions[i],'\n')

label:  1 	 Question:  manner how did serfdom develop in and then leave russia ? 

label:  2 	 Question:  cremat what films featured the character popeye doyle ? 

label:  1 	 Question:  manner how can i find a list of celebrities ' real names ? 

label:  2 	 Question:  animal what fowl grabs the spotlight after the chinese year of the monkey ? 

label:  0 	 Question:  exp what is the full form of .com ? 



#### <span style="color:red">**Question 1.2**</span> 
**What is the purpose of `self.train_numeral_data`? Write your code to print out the first five questions in the numeric format with labels in the training set.**

#Your answer here



**Ans:**
self.train_numeral_data initially is used to convert each text in each of the questions into a sequence of integers. It then pads all the sequences to the same length. Since padding='post', the 0's will get added after the sequence.

In [6]:
#Your code here
print(dm.train_numeral_data[0:5])

[[  29    8   19 3497 2219    5   16  433  814  990    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [  32    2  815  619    1  148 1255 3498    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0

#### <span style="color:red">**Question 1.3**</span> 
**What is the purpose of two dictionaries: `self.word2idx` and `self.idx2word`? Write your code to print out the first five key-value pairs of those dictionaries.**

#Your answer here



**Ans:**
self.word2idx is a dictionary which is used to store the words in train_set as keys and the index value of those text as its values, whereas self.idx2word is used to store all the index value of all the words as keys and the words are stored as corresponding values.

In [7]:
#Your code here
key_storage = []
for keys in dm.word2idx:
    key_storage.append(keys)
del key_storage[5:len(key_storage)]
print('In word2idx.....\n')
for keys in key_storage:
    print('key: ',keys, '\t', 'value: ',dm.word2idx[keys],'\n')
print('In idx2word.....\n')
for i in range(1,6):
    print('key: ',i,'\t','value: ',dm.idx2word[i],'\n')

In word2idx.....

key:  the 	 value:  1 

key:  what 	 value:  2 

key:  is 	 value:  3 

key:  of 	 value:  4 

key:  in 	 value:  5 

In idx2word.....

key:  1 	 value:  the 

key:  2 	 value:  what 

key:  3 	 value:  is 

key:  4 	 value:  of 

key:  5 	 value:  in 



#### <span style="color:red">**Question 1.4**</span> 
**What is the purpose of `self.tf_train_set`? Write your code to print out the first five items of `self.tf_train_set`.**

#Your answer here


**Ans:**
self.tf_train_set is used to convert the values of self.train_numeral_data and self.train_numeral_labels into a TensorSliceDataset.

In [8]:
#Your code here
tensor_store = []
for each in dm.tf_train_set:
    tensor_store.append(each)
del tensor_store[5:]
print('The first five elements of self.tf_train_set are....\n')
print(tensor_store)
print('\nMuch cleaner output will be...\n')
for each in tensor_store:
    tf.print(each)

The first five elements of self.tf_train_set are....

[(<tf.Tensor: shape=(100,), dtype=int32, numpy=
array([  29,    8,   19, 3497, 2219,    5,   16,  433,  814,  990,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0])>, <tf.Tensor: shape=(), dtype=int64, numpy=1>), (<tf.Tensor: shape=(100,), dtype=int32, numpy=
array([  32,    2,  815,  619,    1,  148, 1255, 3498,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0

#### <span style="color:red">**Question 1.5**</span> 
**What is the purpose of `self.tf_valid_set`? Write your code to print out the first five items of `self.tf_valid_set`.**

#Your answer here


**Ans:**
self.tf_train_set is used to tune the hyperparameters/check generalisation.

In [9]:
#Your code here
tensor_store = []
for each in dm.tf_valid_set:
    tensor_store.append(each)
del tensor_store[5:]
print('The first five elements of self.tf_valid_set are....\n')
print(tensor_store)
print('\nMuch cleaner output will be...\n')
for each in tensor_store:
    tf.print(each)

The first five elements of self.tf_valid_set are....

[(<tf.Tensor: shape=(100,), dtype=int32, numpy=
array([  38,   12,  279,    1,   33, 2178,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0])>, <tf.Tensor: shape=(), dtype=int64, numpy=3>), (<tf.Tensor: shape=(100,), dtype=int32, numpy=
array([  27,    2, 6443,  584,   27,   55,    1, 6444,  158, 6445,    1,
       6446,  158,   69, 6447,    0,    0,    0,    0,    0

## <span style="color:#0b486b">Part 2: Using Word2Vect to transform texts to vectors </span>

In this part, you will be assessed on how to use a pretrained Word2Vect model for realizing a machine learning task. Basically, you will use this pretrained Word2Vect to transform the questions in the above dataset stored in the *data manager object dm* to numeric form for training a Support Vector Machine in sckit-learn.  

In [10]:
import gensim.downloader as api
from gensim.models import Word2Vec
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

#### <span style="color:red">**Question 2.1**</span> 
**Write code to download the pretrained model *glove-wiki-gigaword-100*. Note that this model transforms a word in its dictionary to a $100$ dimensional vector.** 

In [11]:
word2vect = api.load('glove-wiki-gigaword-100')

#### <span style="color:red">**Question 2.2**</span> 

**Write code for the function *get_word_vector(word, model)* used to transform a word to a vector using the pretrained Word2Vect model *model*. Note that for a word not in the vocabulary of our *word2vect*, you need to return a vector $0$ with 100 dimensions.**

In [12]:
def get_word_vector(word, model):
    try:
        vector = model[word]
    except: #word not in the vocabulary
        vector = np.zeros(shape=(100,))
    return vector

## <span style="color:#0b486b">Part 3: Text CNN for sequence modeling and neural embedding </span>

#### <span style="color:red">**Question 3.1**</span> 

**In what follows, you are required to complete the code for Text CNN for sentence classification. The paper of Text CNN can be found at this [link](https://www.aclweb.org/anthology/D14-1181.pdf). Here is the description of the Text CNN you need to construct.**
- There are three attributes (properties or instance variables): *embed_size, state_size, data_manager*.
  - `embed_size`: the dimension of the vector space for which the words are embedded to using the embedding matrix.
  - `state_size`: the number of filters used in *Conv1D* (reference [here](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv1D)).
  - `data_manager`: the data manager to store information of the dataset.
- The detail of the computational process is as follows:
  - Given input $x$, we embed $x$ using the embedding matrix to obtain an $3D$ tensor $[batch\_size \times vocab\_size \times embed\_size]$ as $h$.
  - We feed $h$ to three Convd 1D layers, each of which has $state\_size$ filters, padding=same, activation= relu, and $kernel\_size= 3, 5, 7$ respectively to obtain $h1, h2, h3$. Note that each $h1, h2, h3$ is a 3D tensor with the shape $[batch\_size \times output\_size \times state\_size]$.
  - We then apply *GlobalMaxPool1D()* (reference [here](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GlobalMaxPool1D)) over $h1, h2, h3$ to obtain 2D tensors stored in $h1, h2, h3$ again.
  - We then concatenate three 2D tensors $h1, h2, h3$ to obtain $h$. Note that you need to specify the axis to concatenate.
  - We finally build up one dense layer on the top of $h$ for classification.
  

In [13]:
class TextCNN:
    def __init__(self, embed_size= 128, state_size=16, data_manager=None):
        self.data_manager = data_manager
        self.embed_size = embed_size
        self.state_size = state_size
    
    def build(self):
        x = tf.keras.layers.Input(shape=[None])
        h = tf.keras.layers.Embedding(self.data_manager.vocab_size +1, self.embed_size)(x)
        h1 = tf.keras.layers.Conv1D(filters=self.state_size, padding='same', activation='relu', kernel_size=3)(h)#Insert your code here
        h2 = tf.keras.layers.Conv1D(filters=self.state_size, padding='same', activation='relu', kernel_size=5)(h)#Insert your code here
        h3 = tf.keras.layers.Conv1D(filters=self.state_size, padding='same', activation='relu', kernel_size=7)(h)#Insert your code here
        max_pool = tf.keras.layers.GlobalMaxPooling1D()
        h1 = max_pool(h1)
        h2 = max_pool(h2)
        h3 = max_pool(h3)
        h = tf.concat([h1,h2,h3], axis = 0)
        h = tf.keras.layers.Dense(self.data_manager.num_classes, activation='softmax')(h)
        self.model = tf.keras.Model(inputs=x, outputs=h) 
    
    def compile_model(self, *args, **kwargs):
        self.model.compile(*args, **kwargs)
    
    def fit(self, *args, **kwargs):
        return self.model.fit(*args, **kwargs)
    
    def evaluate(self, *args, **kwargs):
        self.model.evaluate(*args, **kwargs)


## <span style="color:#0b486b">Part 4: RNNs for sequence modeling and neural embedding </span>

### <span style="color:#0b486b">4.1. One-directional RNNs for sequence modeling and neural embedding </span> ###

#### <span style="color:red">**Question 4.1.1**</span> 
**In this part, you need to construct an RNN to learn from the dataset of interest. Basically, you are required first to construct the class UniRNN (Uni-directional RNN) with the following requirements:**
- Attribute `data_manager (self.data_manager)`: specifies the data manager used to store data for the model.
- Attribute `cell_type (self.cell_type)`: can receive three values including `basic_rnn`, `gru`, and `lstm` which specifies the memory cells formed a hidden layer.
- `state_sizes (self.state_sizes)` indicates the list of the hidden sizes from the second hidden layers of memory cells. For example, $embed\_size =128$ and $state\_sizes = [64, 64]$ means that you have three hidden layers in your network with hidden sizes of $128, 64$ and $64$ respectively.

**Note that when declaring an embedding layer for the network, you need to set *mask_zero=True* so that the padding zeros in the sentences will be masked and ignored. This helps to have variable length RNNs. For more detail, you can refer to this [link](https://www.tensorflow.org/guide/keras/masking_and_padding).**

In [14]:
class UniRNN:
    def __init__(self, cell_type= 'gru', embed_size= 128, state_sizes= [128, 64], data_manager= None):
        self.cell_type = cell_type
        self.state_sizes = state_sizes
        self.embed_size = embed_size
        self.data_manager = data_manager
        self.vocab_size = self.data_manager.vocab_size +1 
        
    #return the correspoding memory cell
    @staticmethod
    def get_layer(cell_type= 'gru', state_size= 128, return_sequences= False, activation = 'tanh'):
        if cell_type=='gru':
            return tf.keras.layers.GRU(state_size, return_sequences=return_sequences, activation=activation)
        elif cell_type== 'lstm':
            return tf.keras.layers.LSTM(state_size, return_sequences=return_sequences, activation=activation)
        else:
            return tf.keras.layers.SimpleRNN(state_size, return_sequences=return_sequences, activation=activation)
    
    def build(self):
        x = tf.keras.layers.Input(shape=[None])
        h = tf.keras.layers.Embedding(self.vocab_size, self.embed_size, mask_zero=True)(x)
        num_layers = len(self.state_sizes)
        return_sequences = True
        for i in range(num_layers):
            h =  self.get_layer(state_size=self.state_sizes[i], return_sequences = return_sequences, cell_type=self.cell_type)(h)
            return_sequences = False
        h = tf.keras.layers.Dense(dm.num_classes, activation='softmax')(h)
        self.model = tf.keras.Model(inputs=x, outputs=h)
   
    def compile_model(self, *args, **kwargs):
        self.model.compile(*args, **kwargs)
    
    def fit(self, *args, **kwargs):
        return self.model.fit(*args, **kwargs)
    
    def evaluate(self, *args, **kwargs):
        self.model.evaluate(*args, **kwargs)       


#### <span style="color:red">**Question 4.1.2**</span> 
**Run with basic RNN ('basic_rnn') cell with $embed\_size= 128, state\_sizes= [128, 128], data\_manager= dm$.**

In [15]:
uni_rnn = UniRNN(cell_type='basic_rnn',embed_size = 128, state_sizes = [128,128], data_manager=dm)
uni_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
uni_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
uni_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Train for 52 steps, validate for 35 steps
Epoch 1/20
52/52 [==============================] - 19s 374ms/step - loss: 0.5809 - accuracy: 0.8090 - val_loss: 0.5026 - val_accuracy: 0.8661
Epoch 2/20
52/52 [==============================] - 13s 242ms/step - loss: 0.1404 - accuracy: 0.9542 - val_loss: 0.1028 - val_accuracy: 0.9628
Epoch 3/20
52/52 [==============================] - 13s 255ms/step - loss: 0.1273 - accuracy: 0.9655 - val_loss: 0.2673 - val_accuracy: 0.9064
Epoch 4/20
52/52 [==============================] - 13s 249ms/step - loss: 0.0657 - accuracy: 0.9807 - val_loss: 0.4217 - val_accuracy: 0.8775
Epoch 5/20
52/52 [==============================] - 13s 256ms/step - loss: 0.0437 - accuracy: 0.9887 - val_loss: 0.0850 - val_accuracy: 0.9743
Epoch 6/20
52/52 [==============================] - 13s 251ms/step - loss: 0.0598 - accuracy: 0.9841 - val_loss: 0.0814 - val_accuracy: 0.9743
Epoch 7/20
52/52 [==============================] - 13s 250ms/step - loss: 0.0457 - accuracy: 0.9881

#### <span style="color:red">**Question 4.1.3**</span> 
**Run with GRU ('gru') cell with $embed\_size= 128, state\_sizes= [128, 128], data\_manager= dm$.**

In [16]:
uni_rnn = UniRNN(cell_type='gru',embed_size = 128, state_sizes = [128,128], data_manager=dm)
uni_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
uni_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
uni_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Train for 52 steps, validate for 35 steps
Epoch 1/20
52/52 [==============================] - 47s 898ms/step - loss: 0.8961 - accuracy: 0.6498 - val_loss: 0.2492 - val_accuracy: 0.9229
Epoch 2/20
52/52 [==============================] - 25s 488ms/step - loss: 0.1242 - accuracy: 0.9627 - val_loss: 0.1151 - val_accuracy: 0.9638
Epoch 3/20
52/52 [==============================] - 25s 487ms/step - loss: 0.0464 - accuracy: 0.9866 - val_loss: 0.1005 - val_accuracy: 0.9688
Epoch 4/20
52/52 [==============================] - 25s 489ms/step - loss: 0.0238 - accuracy: 0.9939 - val_loss: 0.1117 - val_accuracy: 0.9665
Epoch 5/20
52/52 [==============================] - 25s 490ms/step - loss: 0.0074 - accuracy: 0.9982 - val_loss: 0.1841 - val_accuracy: 0.9633
Epoch 6/20
52/52 [==============================] - 25s 490ms/step - loss: 0.0031 - accuracy: 0.9991 - val_loss: 0.1082 - val_accuracy: 0.9725
Epoch 7/20
52/52 [==============================] - 26s 507ms/step - loss: 0.0229 - accuracy: 0.9969

#### <span style="color:red">**Question 4.1.4**</span> 
**Run with LSTM ('lstm') cell with $embed\_size= 128, state\_sizes= [128, 128], data\_manager= dm$.**

In [17]:
uni_rnn = UniRNN(cell_type='lstm',embed_size = 128, state_sizes = [128,128], data_manager=dm)
uni_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
uni_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
uni_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Train for 52 steps, validate for 35 steps
Epoch 1/20
52/52 [==============================] - 46s 891ms/step - loss: 0.6968 - accuracy: 0.7659 - val_loss: 0.2035 - val_accuracy: 0.9427
Epoch 2/20
52/52 [==============================] - 21s 410ms/step - loss: 0.1246 - accuracy: 0.9639 - val_loss: 0.1031 - val_accuracy: 0.9674
Epoch 3/20
52/52 [==============================] - 20s 383ms/step - loss: 0.0585 - accuracy: 0.9847 - val_loss: 0.0920 - val_accuracy: 0.9702
Epoch 4/20
52/52 [==============================] - 20s 384ms/step - loss: 0.0380 - accuracy: 0.9893 - val_loss: 0.0895 - val_accuracy: 0.9725
Epoch 5/20
52/52 [==============================] - 20s 383ms/step - loss: 0.0177 - accuracy: 0.9960 - val_loss: 0.0998 - val_accuracy: 0.9729
Epoch 6/20
52/52 [==============================] - 20s 384ms/step - loss: 0.0156 - accuracy: 0.9963 - val_loss: 0.1110 - val_accuracy: 0.9748
Epoch 7/20
52/52 [==============================] - 20s 385ms/step - loss: 0.0041 - accuracy: 0.9994

#### <span style="color:red">**Question 4.1.5**</span> 
**Give your own comments about the performance of three memory cells for the dataset of interest as well as what happening during the training process of each cell. Note that there are not right or wrong comments and your comments rely on the status of your training. In addition, some comments and hypothesized assessments of what and why are occurring are useful to obtain the highest score for this question.**

**Ans:**
Out of the three memory cells, basic rnn tends to produce the lowest accuracy and lstm and gru giving the highest accuracies. In Basic RNN, the problem of Gradient vanishing/explosion can occur. In order to address this problem, tanh function is used, which is used to scale the output to (-1,1) at each time step. In GRU, the update gate decides how much each unit updates its state, the reset gate decides which parts of the state will be used to compute the next target state. LSTM consists of two states, long term state and the short term state. The forget gate in LSTM controls how much of the previous long term memory elements should be forgotten, the output gate controls how much of the long term memory elements should be carried on to the next time slice and the input gate controls how much to absorb more from the current information.

### <span style="color:#0b486b">4.2. Bi-directional RNNs for sequence modeling and neural embedding </span> ###

#### <span style="color:red">**Question 4.2.1**</span> 
**In what follow, you will investigate BiRNN. The task is similar to Part 4.1 but you need to write the code for an BiRNN. Note that the function *get_layer(cell_type= 'gru', state_size= 128, return_sequences= False, activation = 'tanh')* has to return the hidden layer with bidirectional memory cells (e.g., Basic RNN, GRU, and LSTM cells).**

**Complete the code of the class *BiRNN*. Note that for the embedding layer you need to set *mask_zero=True*.**

In [18]:
class BiRNN:
    def __init__(self, cell_type= 'gru', embed_size= 128, state_sizes= [128, 64], data_manager= None):
        self.cell_type = cell_type
        self.state_sizes = state_sizes
        self.embed_size = embed_size
        self.data_manager = data_manager
        self.vocab_size = self.data_manager.vocab_size +1
        
    @staticmethod
    def get_layer(cell_type= 'gru', state_size= 128, return_sequences= False, activation = 'tanh'):
        if cell_type=='gru':
            return tf.keras.layers.Bidirectional(tf.keras.layers.GRU(state_size, return_sequences=return_sequences, activation=activation))
        elif cell_type== 'lstm':
            return tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(state_size, return_sequences=return_sequences, activation=activation))
        else:
            return tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(state_size, return_sequences=return_sequences, activation=activation))
    
    def build(self):
        x = tf.keras.layers.Input(shape=[None])
        h = tf.keras.layers.Embedding(self.vocab_size, self.embed_size, mask_zero=True)(x)
        num_layers = len(self.state_sizes)
        return_sequences = True
        for i in range(num_layers):
            h = self.get_layer(state_size=self.state_sizes[i], return_sequences = return_sequences, cell_type=self.cell_type)(h)
            return_sequences = False
        h = tf.keras.layers.Dense(dm.num_classes, activation='softmax')(h)
        self.model = tf.keras.Model(inputs=x, outputs=h)
        
    
    def compile_model(self, *args, **kwargs):
        self.model.compile(*args, **kwargs)
    
    def fit(self, *args, **kwargs):
        return self.model.fit(*args, **kwargs)
    
    def evaluate(self, *args, **kwargs):
        self.model.evaluate(*args, **kwargs)       


#### <span style="color:red">**Question 4.2.2**</span> 
**Run BiRNN for basic RNN ('basic_rnn') cell with $embed\_size= 128, state\_sizes= [128, 128], data\_manager= dm$.**

In [19]:
bi_rnn = BiRNN(cell_type='basic_rnn', embed_size=128, state_sizes=[128,128], data_manager=dm)
bi_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
bi_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
bi_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Train for 52 steps, validate for 35 steps
Epoch 1/20
52/52 [==============================] - 26s 492ms/step - loss: 0.3239 - accuracy: 0.8897 - val_loss: 0.1113 - val_accuracy: 0.9606
Epoch 2/20
52/52 [==============================] - 22s 419ms/step - loss: 0.0667 - accuracy: 0.9774 - val_loss: 0.0877 - val_accuracy: 0.9633
Epoch 3/20
52/52 [==============================] - 22s 416ms/step - loss: 0.0144 - accuracy: 0.9957 - val_loss: 0.1215 - val_accuracy: 0.9642
Epoch 4/20
52/52 [==============================] - 22s 418ms/step - loss: 0.0058 - accuracy: 0.9979 - val_loss: 0.1025 - val_accuracy: 0.9693
Epoch 5/20
52/52 [==============================] - 22s 431ms/step - loss: 2.5191e-04 - accuracy: 1.0000 - val_loss: 0.1064 - val_accuracy: 0.9729
Epoch 6/20
52/52 [==============================] - 22s 422ms/step - loss: 2.2701e-05 - accuracy: 1.0000 - val_loss: 0.1134 - val_accuracy: 0.9725
Epoch 7/20
52/52 [==============================] - 22s 427ms/step - loss: 2.5751e-06 - accu

#### <span style="color:red">**Question 4.2.3**</span> 
**Run BiRNN for GRU ('gru') cell with $embed\_size= 128, state\_sizes= [128, 128], data\_manager= dm$.**

In [20]:
bi_rnn = BiRNN(cell_type='gru', embed_size=128, state_sizes=[128,128], data_manager=dm)
bi_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
bi_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
bi_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Train for 52 steps, validate for 35 steps
Epoch 1/20
52/52 [==============================] - 63s 1s/step - loss: 0.5641 - accuracy: 0.8023 - val_loss: 0.1691 - val_accuracy: 0.9445
Epoch 2/20
52/52 [==============================] - 44s 853ms/step - loss: 0.0786 - accuracy: 0.9743 - val_loss: 0.1232 - val_accuracy: 0.9647
Epoch 3/20
52/52 [==============================] - 44s 851ms/step - loss: 0.0284 - accuracy: 0.9908 - val_loss: 0.1275 - val_accuracy: 0.9656
Epoch 4/20
52/52 [==============================] - 44s 853ms/step - loss: 0.0087 - accuracy: 0.9979 - val_loss: 0.1463 - val_accuracy: 0.9683
Epoch 5/20
52/52 [==============================] - 44s 850ms/step - loss: 0.0053 - accuracy: 0.9988 - val_loss: 0.1562 - val_accuracy: 0.9638
Epoch 6/20
52/52 [==============================] - 45s 866ms/step - loss: 3.1494e-04 - accuracy: 1.0000 - val_loss: 0.1353 - val_accuracy: 0.9720
Epoch 7/20
52/52 [==============================] - 44s 840ms/step - loss: 1.3594e-05 - accuracy: 1

#### <span style="color:red">**Question 4.2.4**</span> 
**Run BiRNN for LSTM ('lstm') cell with $embed\_size= 128, state\_sizes= [128, 128], data\_manager= dm$.**

In [21]:
bi_rnn = BiRNN(cell_type='lstm', embed_size=128, state_sizes=[128,128], data_manager=dm)
bi_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
bi_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
bi_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Train for 52 steps, validate for 35 steps
Epoch 1/20
52/52 [==============================] - 62s 1s/step - loss: 0.6440 - accuracy: 0.7861 - val_loss: 0.1986 - val_accuracy: 0.9399
Epoch 2/20
52/52 [==============================] - 41s 795ms/step - loss: 0.0931 - accuracy: 0.9731 - val_loss: 0.1121 - val_accuracy: 0.9683
Epoch 3/20
52/52 [==============================] - 42s 800ms/step - loss: 0.0327 - accuracy: 0.9908 - val_loss: 0.1173 - val_accuracy: 0.9743
Epoch 4/20
52/52 [==============================] - 42s 804ms/step - loss: 0.0167 - accuracy: 0.9960 - val_loss: 0.2896 - val_accuracy: 0.9541
Epoch 5/20
52/52 [==============================] - 42s 811ms/step - loss: 0.0092 - accuracy: 0.9972 - val_loss: 0.1195 - val_accuracy: 0.9752
Epoch 6/20
52/52 [==============================] - 43s 819ms/step - loss: 0.0018 - accuracy: 0.9994 - val_loss: 0.1500 - val_accuracy: 0.9729
Epoch 7/20
52/52 [==============================] - 41s 790ms/step - loss: 5.9010e-04 - accuracy: 0.999

#### <span style="color:red">**Question 4.2.5**</span> 

**Give your own comments about the performance of three memory cells for the dataset of interest as well as comparing BiRNN to UniRNN in Part 1.**

#Your comments here


**Ans:**
Out of the three variations in the memory cells, gru tends to perform the best followed by lstm and then basic rnn. From observing their behaviour between UniRNN and BiRNN, one can observe that the performance in BiRNN is better for gru, whereas there is a small variation for the accuracy between UniRNN and BiRNN for lstm, while the accuracy of basic rnn remains same for both the models.

--- 
<div style="text-align: center"> <span style="color:black">END OF ASSIGNMENT</span> </div>